# Creates schema subset files. Works with different threads and each thread writes in a separate file. This speeds up the process a lot in comparison to writing in one global file.

In [4]:
import pandas as pd
import gzip
from os import listdir
from os.path import isfile, join
from multiprocessing import Pool, Process, Manager
from tqdm import tqdm
import os

subset_class_path = ""
class_list_file = "/work-ceph/aprimpel/WDC_Extraction_2020/schemaOrgclasses.txt"
class_list_file_pd = pd.read_csv(class_list_file, sep='\t', names=['type','filename'])
class_list_file_dict = pd.Series(class_list_file_pd.filename.values,index=class_list_file_pd.type).to_dict()
input_path = "/work-ceph/aprimpel/WDC_Extraction_2020/6_sorted_quads/json_ld/"
files_json = [f for f in listdir(input_path) if isfile(join(input_path, f))]

output_path = "/work-ceph/aprimpel/WDC_Extraction_2020/7_c_schema_json_full_no_enc_issues/"

for key in class_list_file_dict:
    os.makedirs(output_path+class_list_file_dict[key])

In [5]:
def createSubsets(filepath):
            
    current_url = ""
    types_of_url = set()
    lines_of_url = []
    
    lines_to_append = dict()
    
    for schema_class in class_list_file_dict.keys():
        lines_to_append[schema_class] = []
    
    with  gzip.open(input_path+filepath, 'r') as file_:
        for line_ in file_:
            try:
                line = line_.decode('utf8')
                url = line.split()[-2]
                
            except: 
                print("Bad line")
                continue
            
            if url!= current_url and current_url!="":
                #write lines
                for type_of_url in types_of_url:
                    class_file_name = class_list_file_dict.get(type_of_url)
                    lines_to_append[type_of_url].extend(lines_of_url)

                types_of_url = set()
                lines_of_url = []

                
            lines_of_url.append(line)
            

            if ("<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>" in line):
                for schema_class in class_list_file_dict.keys():
                    for prot in ['http','https']:
                        if ("<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <"+prot+"://"+schema_class+">" in line):
                            types_of_url.add(schema_class)

            
        
            current_url=url
        
        #write the last
        for type_of_url in types_of_url:
            lines_to_append[type_of_url].extend(lines_of_url)
    
    for key in lines_to_append:
        class_file = gzip.open(output_path+class_list_file_dict[key]+"/"+class_list_file_dict[key]+"___"+filepath+".gz", 'a')
        for enc_line in lines_to_append[key]:
            class_file.write(enc_line.encode())
        class_file.close()
    
    #return lines_to_append

In [6]:
pool = Pool(60)
for result in tqdm(pool.imap(func=createSubsets, iterable=files_json), total=len(files_json)):
    pass
    '''
    for key in result:
        class_file = gzip.open(output_path+class_list_file_dict[key]+".gz", 'a')
        for enc_line in result[key]:
            class_file.write(enc_line.encode())
        class_file.close()
    '''

100%|██████████| 5273/5273 [7:59:17<00:00,  5.45s/it]   
